## Using helix_funcs

An example of how to use the helix_funcs module to process the HelixScope datafiles, and generate a master output table.

Process_files should generate a similarly named csv file in the process folder.
combine_processed 

In [1]:
import helix_funcs
import geopandas as gpd
import pandas as pd
# from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#s = gpd.read_file('./data/gadm28_adm1/gadm28_adm1.shp')
#s = s.to_crs(epsg='4326')
s = gpd.read_file("./data/gadm28_adm0_simplified/gadm28_adm0_simplified.shp")  # <--- ADMIN LEVEL 0
#s = gpd.read_file("./data/gadm28_adm1_simplified/gadm28_adm1_simplified.shp")   # <--- ADMIN LEVEL 1

In [3]:
fs = helix_funcs.identify_netcdf_and_csv_files()

In [4]:
fs['nc'][0:2]

['data/CNRS_data/cSoil/orchidee-giss-ecearth.SWL_15.eco.cSoil.nc',
 'data/CNRS_data/cSoil/orchidee-giss-ecearth.SWL_2.eco.cSoil.nc']

In [5]:
%%time
for f in fs['nc'][0:2]:
    helix_funcs.process_file(file=f, shps=s, admin_level=0, verbose=True)

working on  admin0/
Processing 'data/CNRS_data/cSoil/orchidee-giss-ecearth.SWL_15.eco.cSoil.nc'


working on  admin0/
Processing 'data/CNRS_data/cSoil/orchidee-giss-ecearth.SWL_2.eco.cSoil.nc'
CPU times: user 7.55 s, sys: 250 ms, total: 7.8 s
Wall time: 8.61 s


### Speed up 

Looks like we achieve a 34 second execution per file (down from 2 mins) when we use a simplifed set of geometries.



## Investiage areas with no zonal stats

Seems like small areas are failing. Need to investigate best way to handle this.

* check simple shapes produce same number of elements as complex shapes - confimred
* check shapefile has same number of shapes as output file - confirmed
* check simple and complex processed results for same file show good agreement. - confirmed

* Add kwarg for ADMIN0 or Admin1. Have a differnt root folder for both in processed data.
* MOVE alredy processed ADMIN1 level data to the `processed/admin1/` directory - Failed. Columns changed. Will need to recalculate :(
* SIMPLIFY the ADMIN-0 level geometry file as I did with the ADMIN1 file. - confirmed
* RUN ADMIN0 for all data.

Perhaps we should have two tables: ADMIN0 level (based on simplified shapes), and ADMIN1 level.

The ADMIN 0 level makes sense to calculate (mean, min, max, stdev), as those shapes should be large. BUT for ADMIN 1 level, the shapes can be far smaller. In which case it doesnt make sense to calculate those stats for every shape.

* given time, we could work out which shapes it makes sense for (e.g. with counts > 3) and make some distinction.
* with short time (which is the case) it maybe only makes sense to offer mean over those areas (which in most cases will be the mean of 1 cell (i.e. simply the value of the gridcell).


THEREFORE:

1. Process all data to ADMIN 0 (using simplified country shapes)
    * use this as MVP
2. CALCULATE TIME RELATIVE STATISTICS
    * i.e. for datatets with Month distinctions calculate the seasonality
3. Attempt to do the ADMIN 1 level data.

## Deal with outputs

Reduce significant digits. Join together dataframes.

In [6]:
%%time
helix_funcs.combine_processed_results(path='./processed/admin0',
                                      table_name='./master_admin0.csv')  #<-- join all results files together into a master_admin1.csv

Made ./master_admin0.csv: 368 rows of data. 2 sources.
CPU times: user 40 ms, sys: 0 ns, total: 40 ms
Wall time: 54.2 ms


In [7]:
tmp = pd.read_csv("./master_admin0.csv")
tmp.head()

,name_0,iso,variable,swl_info,count,max,min,mean,std,impact_tag,institution,model_long_name,model_short_name,model_taxonomy,is_multi_model_summary,is_seasonal,season,is_monthly,month
0,Norway,NOR,cSoil,1.5,302,22.3,10.2,13.4,2.1,eco,LSCE,ORCHIDEE,orchidee,orchidee-giss-ecearth,False,False,NaN,False,NaN
1,Thailand,THA,cSoil,1.5,218,7.5,2.0,4.1,1.1,eco,LSCE,ORCHIDEE,orchidee,orchidee-giss-ecearth,False,False,NaN,False,NaN
2,"Venezuela, Bolivarian Republic of",VEN,cSoil,1.5,342,11.5,0.6,5.5,2.0,eco,LSCE,ORCHIDEE,orchidee,orchidee-giss-ecearth,False,False,NaN,False,NaN
3,Nigeria,NGA,cSoil,1.5,329,6.3,0.3,2.7,1.2,eco,LSCE,ORCHIDEE,orchidee,orchidee-giss-ecearth,False,False,NaN,False,NaN
4,Argentina,ARG,cSoil,1.5,1221,26.5,0.0,4.5,4.3,eco,LSCE,ORCHIDEE,orchidee,orchidee-giss-ecearth,False,False,NaN,False,NaN


In [8]:
len(tmp)

368

In [ ]:
output_files = helix_funcs.identify_netcdf_and_csv_files('processed/')

In [ ]:
len(output_files['csv'])

In [ ]:
tmp = pd.read_csv(outputs['csv'][-1])

In [ ]:
country_mask = tmp['iso'] == 'ESP'

In [ ]:
tmp = tmp[country_mask].head()

In [ ]:
admin_mask = tmp['id_1'] == 1

In [ ]:
tmp['id_1'].unique()

## Test plots

We need to make an easy way to preview how a specific variables choropleths will appear.
This means read a file, and make a preview plot using geopandas.


In [ ]:
output_files = helix_funcs.identify_netcdf_and_csv_files('processed/')

In [ ]:
helix_funcs.map_file_by_iso(f=output_files['csv'][10], s=s, iso='FRA')

## SIMPLIFY LIFE!

Before passing the polygons for zonal analysis, they should be simplified. 
Looks like the call should be `stest.geometry.simplify(0.2, preserve_topology=False)`

The geometry files themsevels should be simplified, and read in already done. I have another notebook to simply (and correct) these data.

In [ ]:
stest.geometry.simplify(0.2, preserve_topology=False).plot()

In [ ]:
stest = s[s['iso'] == 'BRA']
stest.geometry.plot()

In [ ]:
stest.geometry.simplify(0.2, preserve_topology=True).plot()

## Next Step

Work out problem of small shapes (probably will need to have a logical test for small shapes, and buffer the geom before zonal stats are calculated). Perhaps this should even be done prior to any loop, initially when the shapes are calculaed.

First can look and see what admins are absent from file (to see where the problem lies).

Test with simplifed shapes.

#### REDUCE SIZE OF FINAL DATA BY removing significant digits from values
e.g. 10.6466969914 should be converted to simply 10.6





In [ ]:
def combine_processed_results(path='./processed/admin1',
                              table_name="./master_admin1.csv"):
    """Combine all the csv files in the path (e.g. all processed files)
    into a single master table
    """
    output_files = identify_netcdf_and_csv_files(path)
    frames = [pd.read_csv(csv_file) for csv_file in output_files['csv']]
    master_table = pd.concat(frames)
    master_table.to_csv(table_name, index=False)
    print("Made {0}: {1:,g} rows of data. {2:,g} sources.".format(table_name,
                                                        len(master_table),
                                                        len(output_files['csv'])
                                                                 ))
    return


In [ ]:
path='./processed'
output_files = helix_funcs.identify_netcdf_and_csv_files(path)
frames = [pd.read_csv(csv_file) for csv_file in output_files['csv'][0:10]]

In [ ]:
for  n, frame in enumerate(frames):
    print(n, min(frames[0]['min']), max(frames[0]['min']), min(frames[0]['max']), max(frames[0]['max']), )

In [ ]:
frames[0].head()

In [ ]:
test = frames[0].round(1)

In [ ]:
admin_level = 0

if admin_level == 0:
    admin_prefix = 'admin0/'
elif admin_level == 1:
    admin_prefix = 'admin1/'
else:
    raise ValueError("admin_level kwarg must be either 0 or 1")
print(admin_prefix)